<a href="https://colab.research.google.com/github/mvkhachaturyan/First/blob/main/Parser420.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium beautifulsoup4 pandas
!apt update
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [515 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.0 kB]
Get:10 http://security.u

In [ ]:
#Блок для определения параметров выгрузки
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import csv

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

URL = "https://www.stoloto.ru/4x20/archive"
results = []

driver.get(URL)

max_clicks = 4
current_clicks = 0

with open('lottery_results.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["date", "draw_number", "numbers", "is_jackpot_won", "jackpot_text", "prize"])
    writer.writeheader()

    try:
        while current_clicks < max_clicks:
            draws = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".elem .main")))
            for draw in draws:
                draw_date = draw.find_element(By.CSS_SELECTOR, ".draw_date").get_attribute("title")
                draw_number = draw.find_element(By.CSS_SELECTOR, ".draw a").text
                numbers_zones = draw.find_elements(By.CSS_SELECTOR, ".numbers_wrapper .zone")
                numbers = []
                for zone in numbers_zones:
                    numbers.extend([num.text for num in zone.find_elements(By.TAG_NAME, "b")])

                prize_info = draw.find_element(By.CSS_SELECTOR, ".prize").get_attribute('outerHTML')
                is_jackpot_won = 'with_jackpot' in prize_info
                jackpot_text = "Суперприз разыгран" if is_jackpot_won else ""
                prize = draw.find_element(By.CSS_SELECTOR, ".prize").text.replace(jackpot_text, "").replace("\n", " ").strip().replace("Суперприз разыгран", "")

                writer.writerow({
                    "date": draw_date,
                    "draw_number": draw_number,
                    "numbers": " ".join(numbers),
                    "is_jackpot_won": is_jackpot_won,
                    "jackpot_text": jackpot_text,
                    "prize": prize
                })

            show_more_btn = driver.find_elements(By.CSS_SELECTOR, ".more .pseudo")
            if not show_more_btn:
                break
            driver.execute_script("arguments[0].click();", show_more_btn[0])
            sleep(2)
            current_clicks += 1
    finally:
        driver.quit()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import csv
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

URL = "https://www.stoloto.ru/4x20/archive"

driver.get(URL)

filename = 'lottery_results.csv'

with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["date", "draw_number", "numbers", "is_jackpot_won", "jackpot_text", "prize"])
    writer.writeheader()

    try:
        while True:
            draws = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".elem .main")))
            for draw in draws:
                draw_date = draw.find_element(By.CSS_SELECTOR, ".draw_date").get_attribute("title")
                draw_number = draw.find_element(By.CSS_SELECTOR, ".draw a").text

                # If we have reached draw 1, exit the loop
                if int(draw_number) <= 1:
                    break

                numbers_zones = draw.find_elements(By.CSS_SELECTOR, ".numbers_wrapper .zone")
                numbers = []
                for zone in numbers_zones:
                    numbers.extend([num.text for num in zone.find_elements(By.TAG_NAME, "b")])

                prize_info = draw.find_element(By.CSS_SELECTOR, ".prize").get_attribute('outerHTML')
                is_jackpot_won = 'with_jackpot' in prize_info
                jackpot_text = "Суперприз разыгран" if is_jackpot_won else ""
                prize = draw.find_element(By.CSS_SELECTOR, ".prize").text.replace(jackpot_text, "").replace("\n", " ").strip().replace("Суперприз разыгран", "")

                writer.writerow({
                    "date": draw_date,
                    "draw_number": draw_number,
                    "numbers": " ".join(numbers),
                    "is_jackpot_won": is_jackpot_won,
                    "jackpot_text": jackpot_text,
                    "prize": prize
                })

            # If we have reached draw 1, exit the loop
            if int(draw_number) <= 1:
                break

            show_more_btn = driver.find_elements(By.CSS_SELECTOR, ".more .pseudo")
            if not show_more_btn:
                break
            driver.execute_script("arguments[0].click();", show_more_btn[0])
            sleep(2)
    finally:
        driver.quit()

# Removing duplicates
df = pd.read_csv(filename)
df.drop_duplicates(subset='draw_number', keep='first', inplace=True)
df.to_csv(filename, index=False)
